In [1]:
# Run in python console
# nltk for removing stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/gyul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [7]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [8]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [9]:
import pandas as pd

In [10]:
df = pd.read_json('goodreads_reviews_fantasy_paranormal.json', lines=True)
print(df.review_text.unique())
df.head()

['This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though. \n But what would happen if our SETI stations received a message - if we found someone was out there - and the person monitoring and answering the signal on our side was

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,5577844,52c8ac49496c153e4a97161e36b2db55,5,A beautiful story. Neil Gaiman is truly a uniq...,Wed Sep 24 09:29:29 -0700 2014,Wed Oct 01 00:31:56 -0700 2014,Tue Sep 30 00:00:00 -0700 2014,Sun Sep 21 00:00:00 -0700 2014,5,1
2,8842281e1d1347389f2ab93d60773d4d,17315048,885c772fb033b041f42d57cef5be0a43,5,Mark Watney is a steely-eyed missile man. A ma...,Sat Apr 05 09:30:53 -0700 2014,Wed Mar 22 11:33:10 -0700 2017,Mon Aug 25 00:00:00 -0700 2014,Sat Aug 16 00:00:00 -0700 2014,25,5
3,8842281e1d1347389f2ab93d60773d4d,13453029,46a6e1a14e8afc82d221fec0a2bd3dd0,4,A fun fast paced book that sucks you in right ...,Tue Dec 04 11:12:22 -0800 2012,Sat Jul 26 11:43:28 -0700 2014,Tue Jul 08 00:00:00 -0700 2014,Wed Jul 02 00:00:00 -0700 2014,5,1
4,8842281e1d1347389f2ab93d60773d4d,13239822,a582bfa8efd69d453a5a21a678046b36,3,"This book has a great premise, and is full of ...",Mon Jul 02 16:04:16 -0700 2012,Wed Mar 22 11:32:20 -0700 2017,Wed Aug 15 00:00:00 -0700 2012,Sun Aug 12 00:00:00 -0700 2012,7,0


In [11]:
# Convert to list
data = df.review_text.values.tolist()

pprint(data[:1])

['This is a special book. It started slow for about the first third, then in '
 'the middle third it started to get interesting, then the last third blew my '
 'mind. This is what I love about good science fiction - it pushes your '
 'thinking about where things can go. \n'
 ' It is a 2015 Hugo winner, and translated from its original Chinese, which '
 "made it interesting in just a different way from most things I've read. For "
 'instance the intermixing of Chinese revolutionary history - how they kept '
 'accusing people of being "reactionaries", etc. \n'
 ' It is a book about science, and aliens. The science described in the book '
 'is impressive - its a book grounded in physics and pretty accurate as far as '
 'I could tell. Though when it got to folding protons into 8 dimensions I '
 'think he was just making stuff up - interesting to think about though. \n'
 ' But what would happen if our SETI stations received a message - if we found '
 'someone was out there - and the person 

In [12]:
import random

# Sample a subset of your data
subset_data = random.sample(data, k=10000)  # Adjust the sample size as needed
print(subset_data[:1])

['Even better than the Wattpad original with a more satisfying end than Lanyon usually provides. \n Now where is Spring?']


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(subset_data))

print(data_words[:1])

[['even', 'better', 'than', 'the', 'wattpad', 'original', 'with', 'more', 'satisfying', 'end', 'than', 'lanyon', 'usually', 'provides', 'now', 'where', 'is', 'spring']]


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=2, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['even', 'better_than', 'the', 'wattpad', 'original', 'with', 'more', 'satisfying', 'end', 'than', 'lanyon', 'usually', 'provides', 'now', 'where', 'is', 'spring']


In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['even', 'well', 'wattpad', 'original', 'satisfying', 'end', 'lanyon', 'usually', 'provide', 'spring']]


In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('end', 1),
  ('even', 1),
  ('lanyon', 1),
  ('original', 1),
  ('provide', 1),
  ('satisfying', 1),
  ('spring', 1),
  ('usually', 1),
  ('wattpad', 1),
  ('well', 1)]]

In [19]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.077*"story" + 0.026*"world" + 0.017*"author" + 0.016*"new" + '
  '0.015*"novel" + 0.014*"character" + 0.013*"fantasy" + 0.012*"work" + '
  '0.011*"set" + 0.011*"reader"'),
 (1,
  '0.079*"description" + 0.065*"world_builde" + 0.065*"pull" + 0.063*"race" + '
  '0.062*"check" + 0.057*"free" + 0.052*"slowly" + 0.033*"confuse" + '
  '0.023*"post" + 0.022*"science_fiction"'),
 (2,
  '0.079*"level" + 0.058*"introduction" + 0.031*"deeply" + 0.030*"equally" + '
  '0.030*"wonderfully" + 0.029*"audio" + 0.024*"flawed" + 0.024*"substance" + '
  '0.021*"craft" + 0.016*"shake"'),
 (3,
  '0.021*"find" + 0.018*"take" + 0.018*"life" + 0.015*"people" + 0.013*"leave" '
  '+ 0.013*"go" + 0.012*"try" + 0.012*"help" + 0.011*"begin" + 0.011*"become"'),
 (4,
  '0.000*"sofrer" + 0.000*"tivesse" + 0.000*"pareceu" + 0.000*"provavelmente" '
  '+ 0.000*"quisesse" + 0.000*"resenha" + 0.000*"leitura" + 0.000*"inert" + '
  '0.000*"verdade" + 0.000*"destino"'),
 (5,
  '0.051*"wonder" + 0.044*"consider" + 0.0

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.595581545200423

Coherence Score:  0.40217816604924533


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.459262 -0.112981       1        1  46.347743
0     -0.394802  0.338797       2        1  23.057135
3     -0.414137 -0.220796       3        1  22.052602
7     -0.058097  0.053899       4        1   2.124865
6     -0.016461  0.009431       5        1   1.514674
5      0.000574  0.003313       6        1   1.301299
8      0.019547 -0.011052       7        1   1.102197
1      0.052784 -0.004886       8        1   0.690787
10     0.082106 -0.006052       9        1   0.455882
12     0.098469 -0.005710      10        1   0.273985
2      0.100983 -0.005586      11        1   0.236210
19     0.105070 -0.005265      12        1   0.206786
17     0.104439 -0.005309      13        1   0.147396
15     0.109071 -0.004718      14        1   0.129789
9      0.108786 -0.004769      15        1   0.125599
13     0.108257 -0.004855      16        1   0.122841
11     0.111279 -0.004254      17        1   0.068092
16     0.113636 -0.003225      18        1   0.025056
4      0.113879 -0.002991      19        1   0.008668
18     0.113879 -0.002991      20        1   0.008394, topic_info=        Term        Freq         Total Category  logprob  loglift
13      book  23552.0000  23552.000000  Default  30.0000  30.0000
170    story   9744.0000   9744.000000  Default  29.0000  29.0000
31      read  11531.0000  11531.000000  Default  28.0000  28.0000
117     love   7362.0000   7362.000000  Default  27.0000  27.0000
144   really   6804.0000   6804.000000  Default  26.0000  26.0000
..       ...         ...           ...      ...      ...      ...
25      look      0.0015   1408.421431  Topic20 -10.3113  -4.3675
26    mostly      0.0015    343.072282  Topic20 -10.3113  -2.9552
27      nice      0.0015    492.082480  Topic20 -10.3113  -3.3159
28   perfect      0.0015    650.282053  Topic20 -10.3113  -3.5947
29    pretty      0.0015    979.992635  Topic20 -10.3113  -4.0048

[918 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
567       3  0.999362        able
1056     16  0.936499    absolute
1057      4  0.998406  absolutely
701       1  0.168380      action
701       2  0.831118      action
...     ...       ...         ...
36        2  0.365914       write
256       1  0.016482        year
256       2  0.064898        year
256       3  0.917837        year
1055     12  0.862405         zoe

[473 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 1, 4, 8, 7, 6, 9, 2, 11, 13, 3, 20, 18, 16, 10, 14, 12, 17, 5, 19])

In [27]:
pyLDAvis.save_html(vis, 'lda.html')